老规矩，导入必要的库

In [1]:
import os
import io
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

创建 session

In [2]:
sess = tf.Session()

tf.summary.FileWriter函数的第一个参数表示summary信息写入文件的路径，第二个参数表示我们的graph

In [3]:
board_path = '/tmp/tensorboard'

# Create a visualizer object
summary_writer = tf.summary.FileWriter(board_path, tf.get_default_graph())

# Create tensorboard folder if not exists
if not os.path.exists(board_path):
    os.makedirs(board_path)

算法参数

In [4]:
batch_size = 50
generations = 100

生成训练数据

In [5]:
# Create sample input data
x_data = np.arange(1000)/10.
true_slope = 2.
y_data = x_data * true_slope + np.random.normal(loc=0.0, scale=25, size=1000)

# Split into train/test
train_ix = np.random.choice(len(x_data), size=int(len(x_data)*0.9), replace=False)
test_ix = np.setdiff1d(np.arange(1000), train_ix)
x_data_train, y_data_train = x_data[train_ix], y_data[train_ix]
x_data_test, y_data_test = x_data[test_ix], y_data[test_ix]

构造模型

In [6]:
# Declare placeholders
x_graph_input = tf.placeholder(tf.float32, [None])
y_graph_input = tf.placeholder(tf.float32, [None])

# Declare model variables
m = tf.Variable(tf.random_normal([1], dtype=tf.float32), name='Slope')

# Declare model
output = tf.multiply(m, x_graph_input, name='Batch_Multiplication')

# Declare loss function (L1)
residuals = output - y_graph_input
l1_loss = tf.reduce_mean(tf.abs(residuals), name="L1_Loss")

# Declare optimization function
my_optim = tf.train.GradientDescentOptimizer(0.01)
train_step = my_optim.minimize(l1_loss)

添加scalar类型的统计，scalar 类型适合用于什么场景？

In [7]:
# Visualize a scalar
with tf.name_scope('Slope_Estimate'):
    tf.summary.scalar('Slope_Estimate', tf.squeeze(m))

<tf.Tensor 'Slope_Estimate/Slope_Estimate:0' shape=() dtype=string>

添加 histogram 类型的统计, histogram 类型适合用于什么场景？ 怎么理解 histogram 统计？

In [8]:
# Visualize a histogram (errors)
with tf.name_scope('Loss_and_Residuals'):
    tf.summary.histogram('Histogram_Errors', l1_loss)
    tf.summary.histogram('Histogram_Residuals', residuals)

<tf.Tensor 'Loss_and_Residuals/Histogram_Errors:0' shape=() dtype=string>

<tf.Tensor 'Loss_and_Residuals/Histogram_Residuals:0' shape=() dtype=string>

In [9]:
# Declare summary merging operation
summary_op = tf.summary.merge_all()

In [10]:
# Initialize Variables
init = tf.global_variables_initializer()
sess.run(init)

In [11]:
for i in range(generations):
    batch_indices = np.random.choice(len(x_data_train), size=batch_size)
    x_batch = x_data_train[batch_indices]
    y_batch = y_data_train[batch_indices]
    _, train_loss, summary = sess.run([train_step, l1_loss, summary_op],
                             feed_dict={x_graph_input: x_batch,
                                        y_graph_input: y_batch})
    
    test_loss, test_resids = sess.run([l1_loss, residuals], feed_dict={x_graph_input: x_data_test,
                                                                       y_graph_input: y_data_test})
    
    if (i+1)%10==0:
        print('Generation {} of {}. Train Loss: {:.3}, Test Loss: {:.3}.'.format(i+1, generations, train_loss, test_loss))

    # log_writer = tf.summary.FileWriter(board_path)
    # log_writer.add_summary(summary, i)
    summary_writer.add_summary(summary, i)
    time.sleep(0.5)

Generation 10 of 100. Train Loss: 23.8, Test Loss: 20.7.
Generation 20 of 100. Train Loss: 21.1, Test Loss: 20.6.
Generation 30 of 100. Train Loss: 20.1, Test Loss: 21.6.
Generation 40 of 100. Train Loss: 17.7, Test Loss: 20.7.
Generation 50 of 100. Train Loss: 13.9, Test Loss: 20.7.
Generation 60 of 100. Train Loss: 21.0, Test Loss: 20.6.
Generation 70 of 100. Train Loss: 21.8, Test Loss: 21.0.
Generation 80 of 100. Train Loss: 16.9, Test Loss: 20.6.
Generation 90 of 100. Train Loss: 15.2, Test Loss: 20.6.
Generation 100 of 100. Train Loss: 19.7, Test Loss: 20.8.


In [12]:
#Create a function to save a protobuf bytes version of the graph
def gen_linear_plot(slope):
    linear_prediction = x_data * slope
    plt.plot(x_data, y_data, 'b.', label='data')
    plt.plot(x_data, linear_prediction, 'r-', linewidth=3, label='predicted line')
    plt.legend(loc='upper left')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return(buf)

In [13]:
# Add image to tensorboard (plot the linear fit!)
slope = sess.run(m)
plot_buf = gen_linear_plot(slope[0])

# Convert PNG buffer to TF image
image = tf.image.decode_png(plot_buf.getvalue(), channels=4)

# Add the batch dimension
image = tf.expand_dims(image, 0)

# Add image summary
image_summary_op = tf.summary.image("Linear_Plot", image)
image_summary = sess.run(image_summary_op)
summary_writer.add_summary(image_summary, i)
summary_writer.close()

In [ ]:
http://zangbo.me/2017/06/27/TensorFlow_4/
如果需要train和test统计数据分开，需要创建两个tf.summary.FileWriter
run_metadata = tf.RunMetadata()可以做性能统计
如果需要统计多次结果，最好为每次计算的结果生成一个新的路径名用于保存训练数据

In [6]:
import datetime
import os
save_path = 'tmp'
learning_rate = 0.001
rnn_size = 10
num_layers = 2
embedding_size = 50
batch_size = 100
epochs_to_train = 100
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
filename = 'lr_%.0E,rs_%d,nl_%d,ed_%d,bs_%d,ep_%d,%s' % (
    learning_rate, rnn_size, num_layers, embedding_size, batch_size, epochs_to_train, now)
save_path = os.path.join(save_path, filename)
save_path

'tmp/lr_1E-03,rs_10,nl_2,ed_50,bs_100,ep_100,2017-09-16 16:11:51'